In [34]:
# Set constants
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 1000 # I don't think this actually matters

NUMBER_OF_EPOCHS = 25
LEARNING_RATE = 0.1
MOMENTUM = 0

USE_PNG_FORMAT = False

# The accuracy is measured on the test data, not the train one
# With MOMENTUM=0, LEARNING_RATE=0.01 and BATCH_SIZE=32 I got 76% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.02 and BATCH_SIZE=32 I got 68% accuracy after 9 EPOCHS, 85% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.05 and BATCH_SIZE=32 I got 78% accuracy after 25 EPOCHS
# With MOMENTUM=0, LEARNING_RATE=0.1  and BATCH_SIZE=32 I got 70% accuracy after 25 EPOCHS

In [2]:
import exercise2_config as config
import torch
import numpy as np
import pandas as pd
import os
from datetime import datetime
from PIL import Image


torch.backends.cudnn.enabled = False
torch.manual_seed(round(datetime.timestamp(datetime.now())))


def ReadImages(dir):
  images = []
  for filename in os.listdir(dir):
    image_path = os.path.join(dir, filename)
    image = Image.open(image_path)
    image_as_array = np.array(image.getdata())
    images.append(image_as_array)
    return images


def BuildDataLoadersFromPngFiles():
  for label in range(10):
    dir = os.path.join(config.PNG_TRAIN_DATA_DIR, f'{label}')
    images = ReadImages(dir)
    print(images[0])
    input()

  pass


def BuildDataLoadersFromCsvFiles():
  train_images = pd.read_csv(config.TRAIN_DATA_FILE, header=None)
  test_images = pd.read_csv(config.TEST_DATA_FILE, header=None)

  train_data = np.array(train_images.iloc[:,1:])
  train_labels = np.array(train_images.iloc[:,0])
  test_data = np.array(test_images.iloc[:,1:])
  test_labels = np.array(test_images.iloc[:,0])

  train_data = np.reshape(train_data, (len(train_data), 1, 28, 28))
  train_data = torch.from_numpy(train_data).float()/255
  train_labels = torch.from_numpy(np.array(train_labels))
  train_set = torch.utils.data.TensorDataset(train_data, train_labels)

  test_data = np.reshape(test_data, (len(test_data), 1, 28, 28))
  test_data = torch.from_numpy(test_data).float()/255
  test_labels = torch.from_numpy(np.array(test_labels))
  test_set = torch.utils.data.TensorDataset(test_data, test_labels)
  
  train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE_TEST, shuffle=True)
  return train_loader, test_loader


# We first import the data and convert them to a torch version
def BuildDataLoaders(use_png_format=USE_PNG_FORMAT):
  if use_png_format:
    return BuildDataLoadersFromPngFiles()
  return BuildDataLoadersFromCsvFiles()


train_loader, test_loader = BuildDataLoaders()


In [35]:
%load_ext autoreload
%autoreload 2
import model_task2c
import torch.nn.functional as F


# Something is not quite right and the values become nan
torch.autograd.set_detect_anomaly(True) 


# Initialize classifier stuff
cnn_model = model_task2c.PR_CNN()   # initialization of the model
cnn_model.train()  # activation of the train mode
optimizer = torch.optim.SGD(cnn_model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

def test():
    cnn_model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for _, (data, label) in enumerate(test_loader):
            output = cnn_model(data)
            test_loss += F.nll_loss(output, label, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).sum()

    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def train():
    for epoch in range(NUMBER_OF_EPOCHS):
        print(f'Epoch {epoch + 1}')

        for _, (data, label) in enumerate(train_loader):
            optimizer.zero_grad()

            # forward + backward + optimize
            output = cnn_model(data)
            loss = F.nll_loss(output, label)
            loss.backward()
            optimizer.step()
        
        # Test the model at the end of each epoch
        test()

train()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch 1

Test set: Avg. loss: -0.6361, Accuracy: 6555/10000 (65.55%)

Epoch 2

Test set: Avg. loss: -0.6755, Accuracy: 6797/10000 (67.97%)

Epoch 3

Test set: Avg. loss: -0.6782, Accuracy: 6814/10000 (68.14%)

Epoch 4

Test set: Avg. loss: -0.6835, Accuracy: 6861/10000 (68.61%)

Epoch 5

Test set: Avg. loss: -0.6821, Accuracy: 6844/10000 (68.44%)

Epoch 6

Test set: Avg. loss: -0.6862, Accuracy: 6881/10000 (68.81%)

Epoch 7

Test set: Avg. loss: -0.6896, Accuracy: 6918/10000 (69.18%)

Epoch 8

Test set: Avg. loss: -0.6910, Accuracy: 6919/10000 (69.19%)

Epoch 9

Test set: Avg. loss: -0.6911, Accuracy: 6919/10000 (69.19%)

Epoch 10

Test set: Avg. loss: -0.6948, Accuracy: 6962/10000 (69.62%)

Epoch 11

Test set: Avg. loss: -0.6957, Accuracy: 6971/10000 (69.71%)

Epoch 12

Test set: Avg. loss: -0.6963, Accuracy: 6975/10000 (69.75%)

Epoch 13

Test set: Avg. loss: -0.6970, Accuracy: 6987/10000 (69.87%)